Esta notebook carga el .JSON creado por pptx_cierre_a_json (input) y genera un excel con la data ordenada y legible (output) LLAMANDO A GPT.
No anda perfecto, a veces no puede convertir la respuesta de GPT (un str) a un .csv por problemas de shape y estilo.

In [ ]:
import openai
import pandas as pd
import os
import json
from io import StringIO

from dotenv import load_dotenv

In [ ]:
openai.api_type = os.environ['API_TYPE']
openai.api_version = os.environ['API_VERSION']
openai.azure_endpoint = os.environ['API_BASE']
openai.api_key = os.environ['API_KEY']
GPT_ENGINE = os.environ['GPT_ENGINE']

In [53]:
# Load JSON data from a file
with open('../Data/2024-I+D_Ficha Registro Iniciativas.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Ignore the first 2 and last 2 elements
filtered_data = data[3:-1]

In [62]:
def gpt_json_csv(query): 
    """
    Envía una consulta a un modelo de lenguaje (GPT-4o) y devuelve la respuesta generada.
    En content específica qué debe devolver y provee ejemplos.
    Esta función se focaliza en extraer info de un json

    Parámetros:
    ----------
    json : dict json
        info extraida de una ppt en formato json

    Retorno:
    -------
    str
        La respuesta generada por el modelo de lenguaje, basada en la consulta proporcionada.

    """
    content=f"""Eres un modelo de lenguaje especializado en obtener información detallada de un JSON generado a partir del contenido de un PowerPoint.
    Tu tarea es ayudar a los usuarios a ordenar la información relevante para cada slide_number. Cada slide_number contiene la información en distinto orden.
    Es necesario que estructures tu respuesta de forma ordenada como un CSV listo para transformar en un dataframe. NO INCLUYAS LA PALABRA CSV.
    
    Instrucciones:
    Tenes que identificar y extrar la siguiente información, que luego será incrustrada en un data frame.
    Nombre de la iniciativa, Link de la presentación, link del código, link de la demo, link a la documentacionn,
    tecnologías aplicadas, fecha de inicio, fecha de fin, cantidad de semanas, equipo, tipo de cliente, detalle del cliente,
    interlocutor, celula, documentación validada, comentarios adicionales.
    Quiero que el output sea uno solo, estructurado así, donde cada row es una slide_number y los nombres de las columnas son los siguientes:
    'Proyecto': [valores],
    'Equipo': [valores],
    'Fecha_Inicio': [valores],
    'Fecha_Cierre': [valores],
    'Duración_Semanas': [valores],
    'Industria': [valores],
    'Tipo_Cliente': [valores],
    'Detalle_Cliente': [valores],
    'Interlocutor': [valores],
    'Celula': [valores],
    'Docu_Validada': [valores],
    'Tecnologias': [valores],
    'Comentarios': [valores],
    'Link_Presentacion': [valores],
    'Link_Demo': [valores],
    'Link_Codigo': [valores],
    'Link_Documentacion': [valores],
    """
    
    completion = openai.chat.completions.create(
        model=GPT_ENGINE,
        messages=[
            {"role": "system", "content": content},
            {"role": "user", "content": json.dumps(query)},
        ],
        temperature=0.1,
    )
    response = completion.choices[0].message.content
    return response 

In [97]:
rta=gpt_json_csv(filtered_data[0:10])


In [98]:
print(rta)

'Proyecto','Equipo','Fecha_Inicio','Fecha_Cierre','Duración_Semanas','Industria','Tipo_Cliente','Detalle_Cliente','Interlocutor','Celula','Docu_Validada','Tecnologias','Comentarios','Link_Presentacion','Link_Demo','Link_Codigo','Link_Documentacion'
'Asistentes Bancarios','Adrian Cardozo / FS Mariano Lopez / FS (UNQUI) Juan Garcia / FS (UNQUI) Joaquin Brito / FS','17/07/2024','20/09/2024','8 semanas','No aplica','Externo','Miguel Bilello','Miguel Bilello','Conversacional','Si','OpenAi , Python, React , Celery , Azure Storage, Chroma db , Bot Framework, MaterialUI , Nest','','https://publicisgroupe.sharepoint.com/:p:/r/sites/ID-Equipo/Shared%20Documents/General/Proyectos%20I+D%202024/A)%20Proyectos%20con%20Asistentes/A1_Asistentes%20Conversacionales/06_CONV_NormativaBCRA_closed/Presentacion_Asistente_BCRA.pptx?d=wa1ddc10360d7490d950bd1a0f8e2f1c7&csf=1&web=1&e=N7zbd6','https://publicisgroupe.sharepoint.com/:v:/r/sites/ID-Equipo/Shared%20Documents/General/Proyectos%20I+D%202024/A)%20Proyec

In [100]:
# Use StringIO to simulate reading from a file
csv_file_like = StringIO(rta)

In [101]:
# Read the CSV string into a pandas DataFrame
df = pd.read_csv(csv_file_like)

# Display the DataFrame
print(df)
df.to_excel('output_proyectos_GPT.xlsx', index=False)

                                                                                                                                                                                                             'Proyecto'  \
'Asistentes Bancarios'                     'Adrian Cardozo / FS Mariano Lopez / FS (UNQUI)... '17/07/2024' '20/09/2024' '8 semanas'  'No aplica'                          'Externo' 'Miguel Bilello'   'Miguel Bilello'   
'AI Testing conversacional'                'AdrIán Cardozo / FS Dayana Monsalve / DS Ezequ... '24/06/2024' '8/11/2024'  '16 semanas' 'No aplica'                          'Interno' 'Testing'         'Adriana Gimenez'   
'Formulario con audios (etapa 2)'          'Bilocopetiuc Pablo / FullStack Amarilla Leandr... '23/09/2024' '1/11/2024'  '6 semanas'  'No aplica'                          'i+D'     'i+D'                         'i+D'   
'Comparativo de asistentes'                'Pablo Bilocopetiuc / FS Leandro Amarilla / FS ... '22/07/2024' '09/08/2024' '3 s